In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
import joblib


In [335]:
data = pd.read_csv("k2pandc_2025.09.22_11.36.03.csv")
data.head()

,default_flag,disposition,disp_refname,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,...,sy_gaiamag,sy_icmag,sy_tmag,sy_kepmag,rowupdate,pl_pubdate,releasedate,pl_nnotes,k2_campaigns,k2_campaigns_num
0,0,CONFIRMED,Espinoza et al. 2016,1,1,0,0,Transit,2016,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,...,10.8644,NaN,10.4021,11.040,2018-04-25,2018-03,2018-02-15,1,4,1.0
1,0,CONFIRMED,Espinoza et al. 2016,1,1,0,0,Transit,2016,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,...,10.8644,NaN,10.4021,11.040,2018-04-25,2016-10,2016-07-28,1,4,1.0
2,1,CONFIRMED,Espinoza et al. 2016,1,1,0,0,Transit,2016,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,...,10.8644,NaN,10.4021,11.040,2018-04-25,2017-03,2018-04-26,1,4,1.0
3,0,CANDIDATE,Livingston et al. 2018,1,0,0,0,Transit,2018,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,...,11.3995,NaN,10.7606,11.363,2018-02-15,2018-03,2018-02-15,0,10,1.0
4,1,CANDIDATE,Livingston et al. 2018,1,0,0,0,Transit,2018,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,...,11.3995,NaN,10.7606,11.363,2018-08-02,2018-08,2018-08-02,0,10,1.0


In [336]:
data.shape # i have 112 features way to much 

(3992, 112)

In [337]:
values_null = data.isna().mean() * 100 
type(values_null)


pandas.core.series.Series

In [338]:
def null_elimination(
    null_values: pd.Series, threshold: int, names: list[str]
) -> list[str]:
    """_summary_

    Args:
        null_values (pd.Series): null pandas series (data.isna().mean())
        threshold (int): our allowed null percent in each feature

    Returns:
        list[str]: features to eliminant
    """
    selected_features = []
    for i in range(len(null_values)):
        if null_values[i] > threshold:
            selected_features.append(names[i])

    return selected_features

In [339]:
null_eliminated_values = null_elimination(null_values=values_null,threshold=0.4,names=data.columns)
len(null_eliminated_values)

C:\Users\Samy Adel\AppData\Local\Temp\ipykernel_14096\2301728522.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if null_values[i] > threshold:


66

we have a lot of columns that has more that 40% null values which is not ideal for any machine learning model  

In [340]:
def drop_unwanted_features(features: list[str], data: pd.DataFrame) -> pd.DataFrame:
    """_summary_

    Args:
        features (list[str]): features i don't want
        data (pd.DataFrame): the original data

    Returns:
        pd.DataFrame: copy of the original data without the non-wanted features
    """
    return_data = data.copy()
    return_data.drop(labels=features, axis=1,inplace=True)
    return return_data

In [341]:
df = drop_unwanted_features(features=null_eliminated_values,data=data)
df.head()

,default_flag,disposition,disp_refname,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,...,elat,elon,sy_tmag,sy_kepmag,rowupdate,pl_pubdate,releasedate,pl_nnotes,k2_campaigns,k2_campaigns_num
0,0,CONFIRMED,Espinoza et al. 2016,1,1,0,0,Transit,2016,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,...,1.31208,56.28872,10.4021,11.040,2018-04-25,2018-03,2018-02-15,1,4,1.0
1,0,CONFIRMED,Espinoza et al. 2016,1,1,0,0,Transit,2016,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,...,1.31208,56.28872,10.4021,11.040,2018-04-25,2016-10,2016-07-28,1,4,1.0
2,1,CONFIRMED,Espinoza et al. 2016,1,1,0,0,Transit,2016,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,...,1.31208,56.28872,10.4021,11.040,2018-04-25,2017-03,2018-04-26,1,4,1.0
3,0,CANDIDATE,Livingston et al. 2018,1,0,0,0,Transit,2018,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,...,-4.22377,186.01709,10.7606,11.363,2018-02-15,2018-03,2018-02-15,0,10,1.0
4,1,CANDIDATE,Livingston et al. 2018,1,0,0,0,Transit,2018,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,...,-4.22377,186.01709,10.7606,11.363,2018-08-02,2018-08,2018-08-02,0,10,1.0


In [342]:
df.shape

(3992, 46)

from hear we will do normal machine learning

In [343]:
y = df["disposition"]
X = df.drop(labels=["disposition"], axis=1)
X.shape, y.shape

((3992, 45), (3992,))

In [344]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3193, 45), (799, 45), (3193,), (799,))

In [345]:
# select categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns
len(categorical_cols), len(numerical_cols)

(17, 28)

In [346]:
y_train.value_counts()

disposition
CONFIRMED         1846
CANDIDATE         1095
FALSE POSITIVE     234
REFUTED             18
Name: count, dtype: int64

basically turn form multi-class classification to a binary classification    

In [ ]:
def replace_values_in_target(
    replace_names: list[str], target: pd.Series, new_value: str
) -> pd.Series:
    """_summary_

    Args:
        replace_names (list[str]): values to replace
        target (pd.Series): the target series
        new_value (str): the new value

    Returns:
        pd.Series: the modified target series
    """
    target_copy = target.copy()
    for name in replace_names:
        target_copy.replace(to_replace=name, value=new_value, inplace=True)
    return target_copy


In [348]:
replace_names = ["FALSE POSITIVE", "REFUTED"]
y_train = replace_values_in_target(
    replace_names=replace_names, target=y_train, new_value="CANDIDATE"
)
y_train.value_counts()

disposition
CONFIRMED    1846
CANDIDATE    1347
Name: count, dtype: int64

In [349]:
categorical_cols

Index(['disp_refname', 'discoverymethod', 'disc_refname', 'disc_pubdate',
       'disc_locale', 'disc_facility', 'disc_telescope', 'disc_instrument',
       'soltype', 'pl_refname', 'sy_refname', 'rastr', 'decstr', 'rowupdate',
       'pl_pubdate', 'releasedate', 'k2_campaigns'],
      dtype='object')

disp_refname , discoverymethod , disc_refname, disc_pubdate, disc_facility, disc_telescope , disc_instrument,soltype,disc_locale ,pl_refname ,
sy_refname , 

decstr ,rastr , rowupdate, pl_pubdate, releasedate, 

In [350]:
for i in categorical_cols:
    print(f"{i} : {X_train[i].nunique()}")

disp_refname : 133
discoverymethod : 3
disc_refname : 141
disc_pubdate : 74
disc_locale : 3
disc_facility : 10
disc_telescope : 10
disc_instrument : 11
soltype : 3
pl_refname : 240
sy_refname : 3
rastr : 1354
decstr : 1354
rowupdate : 170
pl_pubdate : 101
releasedate : 188
k2_campaigns : 32


In [351]:
X_train["discoverymethod"].value_counts() #this one could be useful

discoverymethod
Transit            3173
Radial Velocity      19
Microlensing          1
Name: count, dtype: int64

by a simple manual analysis we found that only k2_campaigns could be of value when we do classification  

In [352]:
numerical_cols

Index(['default_flag', 'sy_snum', 'sy_pnum', 'sy_mnum', 'cb_flag', 'disc_year',
       'rv_flag', 'pul_flag', 'ptv_flag', 'tran_flag', 'ast_flag', 'obm_flag',
       'micro_flag', 'etv_flag', 'ima_flag', 'dkin_flag', 'pl_controv_flag',
       'ttv_flag', 'ra', 'dec', 'glat', 'glon', 'elat', 'elon', 'sy_tmag',
       'sy_kepmag', 'pl_nnotes', 'k2_campaigns_num'],
      dtype='object')

default_flag, sy_snum ,sy_mnum, cb_flag, disc_year, pul_flag,ptv_flag, ast_flag , obm_flag , 

In [353]:
# get the varaince of numerical features
X_train[numerical_cols].var()

default_flag           0.247826
sy_snum                0.065180
sy_pnum                1.910758
sy_mnum                0.000000
cb_flag                0.000000
disc_year              4.177631
rv_flag                0.185625
pul_flag               0.000000
ptv_flag               0.000000
tran_flag              0.005917
ast_flag               0.000000
obm_flag               0.000000
micro_flag             0.000313
etv_flag               0.000000
ima_flag               0.000000
dkin_flag              0.000000
pl_controv_flag        0.000313
ttv_flag               0.043363
ra                  8804.442838
dec                  230.767533
glat                1757.608823
glon                9850.588378
elat                  14.457873
elon                8721.930061
sy_tmag                2.628816
sy_kepmag              2.730833
pl_nnotes              0.673132
k2_campaigns_num       0.335616
dtype: float64

In [354]:
def drop_based_variance(
    threshold: float, data: pd.DataFrame, numerical_cols: list[str]
) -> list[str]:
    """_summary_

    Args:
        threshold (float): the variance threshold
        data (pd.DataFrame): the data
        numerical_cols (list[str]): the numerical columns

    Returns:
        list[str]: features to drop
    """
    to_drop = []
    for col in numerical_cols:
        if data[col].var() < threshold:
            to_drop.append(col)
    return to_drop

In [355]:
unwanted_numerical = drop_based_variance(
    threshold=0.1, data=X_train, numerical_cols=numerical_cols
)
len(unwanted_numerical) , len(numerical_cols), 

(14, 28)

In [356]:
X_train = drop_unwanted_features(features=unwanted_numerical, data=X_train)
X_train.shape

(3193, 31)

In [357]:
# select only k2_campaign from the categorical columns
categorical_cols_to_drop = categorical_cols
X_train = drop_unwanted_features(features=categorical_cols_to_drop, data=X_train)
X_train.shape

(3193, 14)

In [358]:
X_train.head()

,default_flag,sy_pnum,disc_year,rv_flag,ra,dec,glat,glon,elat,elon,sy_tmag,sy_kepmag,pl_nnotes,k2_campaigns_num
1918,0,3,2024,0,129.689446,23.685265,33.43766,201.24412,5.06302,125.94923,6.3687,6.793,0,2.0
2120,0,1,2017,0,134.444140,21.453275,36.91396,205.51379,4.08587,130.79159,13.1214,14.102,0,3.0
1276,0,0,2016,0,199.578024,-9.691870,52.60772,313.87907,-1.32186,201.71886,11.8994,12.471,0,2.0
3534,1,1,2016,1,333.751879,-17.250800,-52.63220,39.93320,-5.97399,329.44981,12.3071,13.087,1,1.0
3235,1,1,2021,0,128.321913,13.661376,28.65234,211.64492,-4.96509,127.21099,12.6649,13.438,0,1.0


In [359]:
X_train.isna().sum()

default_flag         0
sy_pnum              0
disc_year            0
rv_flag              0
ra                   0
dec                  0
glat                 0
glon                 0
elat                 0
elon                 0
sy_tmag              3
sy_kepmag           12
pl_nnotes            0
k2_campaigns_num     1
dtype: int64

In [360]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3193 entries, 1918 to 3750
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   default_flag      3193 non-null   int64  
 1   sy_pnum           3193 non-null   int64  
 2   disc_year         3193 non-null   int64  
 3   rv_flag           3193 non-null   int64  
 4   ra                3193 non-null   float64
 5   dec               3193 non-null   float64
 6   glat              3193 non-null   float64
 7   glon              3193 non-null   float64
 8   elat              3193 non-null   float64
 9   elon              3193 non-null   float64
 10  sy_tmag           3190 non-null   float64
 11  sy_kepmag         3181 non-null   float64
 12  pl_nnotes         3193 non-null   int64  
 13  k2_campaigns_num  3192 non-null   float64
dtypes: float64(9), int64(5)
memory usage: 374.2 KB


In [361]:
se = SimpleImputer(strategy="median")
tt = se.fit_transform(X_train)
X_train = pd.DataFrame(tt, columns=X_train.columns)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3193 entries, 0 to 3192
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   default_flag      3193 non-null   float64
 1   sy_pnum           3193 non-null   float64
 2   disc_year         3193 non-null   float64
 3   rv_flag           3193 non-null   float64
 4   ra                3193 non-null   float64
 5   dec               3193 non-null   float64
 6   glat              3193 non-null   float64
 7   glon              3193 non-null   float64
 8   elat              3193 non-null   float64
 9   elon              3193 non-null   float64
 10  sy_tmag           3193 non-null   float64
 11  sy_kepmag         3193 non-null   float64
 12  pl_nnotes         3193 non-null   float64
 13  k2_campaigns_num  3193 non-null   float64
dtypes: float64(14)
memory usage: 349.4 KB


In [362]:
sa = StandardScaler()
X_train = pd.DataFrame(sa.fit_transform(X_train), columns=X_train.columns)
X_train.head()

,default_flag,sy_pnum,disc_year,rv_flag,ra,dec,glat,glon,elat,elon,sy_tmag,sy_kepmag,pl_nnotes,k2_campaigns_num
0,-0.909208,1.356674,3.160807,-0.571443,-0.546315,1.474131,0.554113,0.046309,1.646740,-0.586455,-3.589057,-3.639956,-0.492121,1.191271
1,-0.909208,-0.090415,-0.264512,-0.571443,-0.495635,1.327180,0.637046,0.089335,1.389714,-0.534597,0.578381,0.791962,-0.492121,2.917885
2,-0.909208,-0.813959,-0.753843,-0.571443,0.198628,-0.723373,1.011444,1.181347,-0.032716,0.224986,-0.175778,-0.197019,-0.492121,1.191271
3,1.099858,-0.090415,-0.753843,1.749955,1.628790,-1.221043,-1.499213,-1.579242,-1.256395,1.592897,0.075835,0.176502,0.726919,-0.535342
4,1.099858,-0.090415,1.692813,-0.571443,-0.560892,0.814172,0.439952,0.151120,-0.991018,-0.572942,0.296651,0.389336,-0.492121,-0.535342


In [363]:
X_train.isna().sum()   

default_flag        0
sy_pnum             0
disc_year           0
rv_flag             0
ra                  0
dec                 0
glat                0
glon                0
elat                0
elon                0
sy_tmag             0
sy_kepmag           0
pl_nnotes           0
k2_campaigns_num    0
dtype: int64

In [364]:
# trans = ColumnTransformer(
#     transformers=[
#         (
#             "num",
#             SimpleImputer(strategy="median"),
#             X_train.select_dtypes(include=["int64", "float64"]).columns,
#         ),
#         (
#             "scaler",
#             StandardScaler(),
#             X_train.select_dtypes(include=["int64", "float64"]).columns,
#         ),
#     ]
# )

In [365]:
# # svc
# svc_pipe = make_pipeline(trans, SVC())
# # logistic regression
# logreg_pipe = make_pipeline(trans, LogisticRegression(max_iter=1000))
# # random forest
# rf_pipe = make_pipeline(trans, RandomForestClassifier())
# # gradient boosting
# gb_pipe = make_pipeline(trans, GradientBoostingClassifier())
# # decision tree
# dt_pipe = make_pipeline(trans, DecisionTreeClassifier())
# # knn
# knn_pipe = make_pipeline(trans, KNeighborsClassifier())
# # naive bayes
# gnb_pipe = make_pipeline(trans, GaussianNB())
# # xgboost
# xgb_pipe = make_pipeline(
#     trans, XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
# )
# # lightgbm
# lgbm_pipe = make_pipeline(trans, LGBMClassifier())

In [366]:
# # training 
# svc_pipe.fit(X_train, y_train)
# logreg_pipe.fit(X_train, y_train)
# rf_pipe.fit(X_train, y_train)
# gb_pipe.fit(X_train, y_train)
# dt_pipe.fit(X_train, y_train)
# knn_pipe.fit(X_train, y_train)
# gnb_pipe.fit(X_train, y_train)
# xgb_pipe.fit(X_train, y_train)
# lgbm_pipe.fit(X_train, y_train)

# # predictions
# svc_pred = svc_pipe.predict(X_train)
# logreg_pred = logreg_pipe.predict(X_train)
# rf_pred = rf_pipe.predict(X_train)
# gb_pred = gb_pipe.predict(X_train)
# dt_pred = dt_pipe.predict(X_train)
# knn_pred = knn_pipe.predict(X_train)
# gnb_pred = gnb_pipe.predict(X_train)
# xgb_pred = xgb_pipe.predict(X_train)
# lgbm_pred = lgbm_pipe.predict(X_train)

# # evaluation
# print("SVC")
# print(classification_report(y_train, svc_pred))
# print("Logistic Regression")
# print(classification_report(y_train, logreg_pred))
# print("Random Forest")
# print(classification_report(y_train, rf_pred))
# print("Gradient Boosting")
# print(classification_report(y_train, gb_pred))
# print("Decision Tree")
# print(classification_report(y_train, dt_pred))
# print("KNN")
# print(classification_report(y_train, knn_pred))
# print("Naive Bayes")
# print(classification_report(y_train, gnb_pred))
# print("XGBoost")
# print(classification_report(y_train, xgb_pred))
# print("LightGBM")
# print(classification_report(y_train, lgbm_pred))

In [367]:
lg = LogisticRegression(max_iter=1000)
lg.fit(X_train, y_train)
lg_prd = lg.predict(X_train)
print(classification_report(y_train, lg_prd))

              precision    recall  f1-score   support

   CANDIDATE       1.00      0.98      0.99      1347
   CONFIRMED       0.98      1.00      0.99      1846

    accuracy                           0.99      3193
   macro avg       0.99      0.99      0.99      3193
weighted avg       0.99      0.99      0.99      3193



In [368]:
lg = SVC(max_iter=1000)
lg.fit(X_train, y_train)
lg_prd = lg.predict(X_train)
print(classification_report(y_train, lg_prd))

e:\Samy\projects\nasa\.venv\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


              precision    recall  f1-score   support

   CANDIDATE       1.00      0.98      0.99      1347
   CONFIRMED       0.98      1.00      0.99      1846

    accuracy                           0.99      3193
   macro avg       0.99      0.99      0.99      3193
weighted avg       0.99      0.99      0.99      3193



In [369]:
lg = LGBMClassifier()
lg.fit(X_train, y_train)
lg_prd = lg.predict(X_train)
print(classification_report(y_train, lg_prd))

[LightGBM] [Info] Number of positive: 1846, number of negative: 1347
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002426 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2083
[LightGBM] [Info] Number of data points in the train set: 3193, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.578140 -> initscore=0.315141
[LightGBM] [Info] Start training from score 0.315141
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
              precision    recall  f1-score   support

   CANDIDATE       1.00      1.00      1.00      1347
   CONFIRMED       1.00      1.00      1.00      1846

    accuracy                           1.00      3193
   macro avg       1.00      1.00      1.00      3193
weighted avg       1.00      1.00      1.00      3193



In [370]:
y_test = replace_values_in_target(
    replace_names=replace_names, target=y_test, new_value="CANDIDATE"
)

X_test = drop_unwanted_features(features=unwanted_numerical, data=X_test)
X_test = drop_unwanted_features(features=categorical_cols_to_drop, data=X_test)
X_test.shape, y_test.shape
X_test = pd.DataFrame(se.transform(X_test), columns=X_test.columns)
X_test = pd.DataFrame(sa.transform(X_test), columns=X_test.columns)
X_test.shape, y_test.shape

((799, 14), (799,))

In [371]:
lg_prd = lg.predict(X_test)
print(classification_report(y_test, lg_prd))

              precision    recall  f1-score   support

   CANDIDATE       1.00      0.99      0.99       337
   CONFIRMED       0.99      1.00      0.99       462

    accuracy                           0.99       799
   macro avg       0.99      0.99      0.99       799
weighted avg       0.99      0.99      0.99       799

